In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import requests
import pandas as pd
import json
from citipy import citipy
import random

# Import API key
from config import api_key

In [2]:
# Base URL required for API call
url = "https://developers.zomato.com/api/v2.1/search?"

# Parameters to run through Query URL
start = np.arange(0,100,20)
count = 20
city = 259
radius = 2500
b = 0

# Latitude and Longitudes of Key Locations:
# Cnr William & Bourke; Franklin & Elizabeth; GPO; Russell & Collins; 
# (Optional: Elizabeth & Flinders)
lat = [-37.8156,-37.8081,-37.8136,-37.8147]
lon = [144.9581,144.9604,144.9631,144.9689]

In [3]:
# Empty list to hold all API calls
response_data = []

In [4]:
# Loop through Zomato API
for a in range(len(start)):
    for b in range(len(lat)):
        query_url = f"{url}entity_id={city}&entity_type=metro&lat={lat[b]}&lon={lon[b]}&radius={radius}&sort=real_distance&apikey={api_key}&start={start[a]}&count={count}"
        response_data.append(requests.get(query_url).json())
        print(start[a], lat[b], lon[b])
    b += 1

0 -37.8156 144.9581
0 -37.8081 144.9604
0 -37.8136 144.9631
0 -37.8147 144.9689
20 -37.8156 144.9581
20 -37.8081 144.9604
20 -37.8136 144.9631
20 -37.8147 144.9689
40 -37.8156 144.9581
40 -37.8081 144.9604
40 -37.8136 144.9631
40 -37.8147 144.9689
60 -37.8156 144.9581
60 -37.8081 144.9604
60 -37.8136 144.9631
60 -37.8147 144.9689
80 -37.8156 144.9581
80 -37.8081 144.9604
80 -37.8136 144.9631
80 -37.8147 144.9689


In [5]:
# Empty list to hold all restaurant names, aggregate rating, hours of operation, type of eatery, price 
restr_names = []
rating = []
cuisine_type = []
price_range = []
restr_lat = []
restr_lon = []
c = 0

# Populate lists with JSON data from Zomato 
for c in range(count):
    for x in range(count):
        restr_names.append(response_data[c]["restaurants"][x]["restaurant"]["name"])
        rating.append(response_data[c]["restaurants"][x]["restaurant"]["user_rating"]["aggregate_rating"])
        cuisine_type.append(response_data[c]["restaurants"][x]["restaurant"]["cuisines"])
        price_range.append(response_data[c]["restaurants"][x]["restaurant"]["price_range"])
        restr_lat.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["latitude"])
        restr_lon.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["longitude"])
    c += 1

In [6]:
len(restr_names)

400

In [188]:
response_data[0]["restaurants"]

[{'restaurant': {'R': {'res_id': 17849428,
    'is_grocery_store': False,
    'has_menu_status': {'delivery': -1, 'takeaway': -1}},
   'apikey': '4ceef556bc96bbbc555b4f3b3ce337d3',
   'id': '17849428',
   'name': 'Doughboys Doughnuts',
   'url': 'https://www.zomato.com/melbourne/doughboys-doughnuts-cbd?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
   'location': {'address': '535 Bourke Street, CBD, Melbourne',
    'locality': 'CBD',
    'city': 'Melbourne',
    'city_id': 259,
    'latitude': '-37.8159507485',
    'longitude': '144.9578592181',
    'zipcode': '3000',
    'country_id': 14,
    'locality_verbose': 'CBD, Melbourne'},
   'switch_to_order_menu': 0,
   'cuisines': 'Coffee and Tea, Desserts',
   'timings': '7 AM to 4:30 PM (Mon-Fri), 9 AM to 4 PM (Sat), Sun Closed',
   'average_cost_for_two': 20,
   'price_range': 1,
   'currency': '$',
   'highlights': ['Cash',
    'Debit Card',
    'Delivery',
    'No Alcohol Available',
    'Credit Card',
    'Takeaway Avail

In [7]:
# Make DataFrame from list data
zomato_df = pd.DataFrame({
    "Name": restr_names,
    "Aggregate Rating": rating,
    "Type": cuisine_type,
    "Price": price_range,
    "Latitude": restr_lat,
    "Longitude": restr_lon,
})

zomato_df.tail(10)

,Name,Aggregate Rating,Type,Price,Latitude,Longitude
390,KFC,2.8,"Fast Food, Fried Chicken",2,-37.8130790491,144.9674558267
391,Trinket,3.8,"Italian, Bar Food",4,-37.8152464747,144.9713259190
392,Nashi - SX2 Southern Cross Lane,3.2,"Healthy Food, Salad, Sandwich",1,-37.8129450231,144.9701108784
393,Swiss Club Restaurant,4.1,"European, Swiss",4,-37.8152666044,144.9713275954
394,Nana Thai Style Hotpot & BBQ,3.6,"Thai, BBQ, Hot Pot",1,-37.8127108744,144.9685803428
395,MoVida Bar De Tapas,4.7,"Spanish, Tapas",4,-37.8167026904,144.9691895396
396,VB Rolls,3.4,"Vietnamese, Sandwich",2,-37.8138408205,144.9665834382
397,Sushi Land,3.3,Sushi,2,-37.8151654258,144.9714000151
398,Final Pass,3.4,Drinks Only,1,-37.8131582462,144.9672231451
399,Pastuso,4.6,"Latin American, Peruvian",4,-37.8158421546,144.9710432813


In [8]:
# Drop duplicates from DataFrame
z_clean = zomato_df.drop_duplicates(subset="Name", keep=False).copy()
z_clean.count()

Name                375
Aggregate Rating    375
Type                375
Price               375
Latitude            375
Longitude           375
dtype: int64

In [9]:
# Change column types of Rating, Latitude and Longitude from strings to numbers (float)
z_clean["Aggregate Rating"] = z_clean["Aggregate Rating"].astype(float)
z_clean["Latitude"] = z_clean["Latitude"].astype(float)
z_clean["Longitude"] = z_clean["Longitude"].astype(float)
z_clean

,Name,Aggregate Rating,Type,Price,Latitude,Longitude
0,Doughboys Doughnuts,4.0,"Coffee and Tea, Desserts",1,-37.815951,144.957859
1,Bluestones Cafe,3.0,"Coffee and Tea, Cafe Food",2,-37.815490,144.958588
2,Sakae,2.7,"Japanese, Sushi",2,-37.815578,144.957547
3,Crisp,3.8,"Healthy Food, Vegetarian, Salad",2,-37.815578,144.957509
4,Pho Melbourne,3.1,Vietnamese,2,-37.815073,144.958033
...,...,...,...,...,...,...
395,MoVida Bar De Tapas,4.7,"Spanish, Tapas",4,-37.816703,144.969190
396,VB Rolls,3.4,"Vietnamese, Sandwich",2,-37.813841,144.966583
397,Sushi Land,3.3,Sushi,2,-37.815165,144.971400
398,Final Pass,3.4,Drinks Only,1,-37.813158,144.967223


In [10]:
# Consolidate eatery types 
z_clean_2 = z_clean.replace({
    'American, Bar Food':'Bar Food',
    'American, Coffee and Tea, Cafe Food': 'Cafe Food',
    'American, Fried Chicken': 'American',
    'American, Pub Food': 'Pub Food',
    'American, Steak':'American',   
    'Asian, Desserts': 'Desserts',
    'Asian, Desserts, Ice Cream': 'Desserts',
    'Asian, Fast Food':'Fast Food',
    'Asian, Fusion': 'Asian',
    'Asian, Indonesian':'Indonesian',
    'Asian, Japanese':'Japanese',
    'Asian, Malaysian, Chinese':'Malaysian',
    'Asian, Sri Lankan, Modern Australian':'Asian',
    'Australian, Contemporary': 'Australian',
    'BBQ, Brazilian, Steak': 'BBQ',
    'BBQ, Pizza, Fast Food': 'BBQ',
    'BBQ, Steak': 'BBQ',
    'Bakery, Coffee and Tea':'Bakery',
    'Bakery, Desserts':'Desserts',
    'Bakery, Fast Food':'Fast Food',
    'Bakery, French':'Bakery',
    'Bakery, Middle Eastern':'Bakery',
    'Bakery, Patisserie':'Bakery',
    'Bakery, Turkish':'Bakery',
    'Bar Food, Asian Fusion':'Bar Food',
    'Bar Food, Asian, Dumplings':'Bar Food',
    'Bar Food, Contemporary':'Bar Food',
    'Bar Food, French':'Bar Food',
    'Bar Food, Middle Eastern':'Bar Food',
    'Bar Food, Modern Australian':'Bar Food',
    'Bar Food, Modern European':'Bar Food',
    'Beverages, Bubble Tea, Tea':'Bubble Tea',
    'Bubble Tea, Asian':'Bubble Tea',
    'Bubble Tea, Cafe Food, Coffee and Tea':'Bubble Tea',
    'Bubble Tea, Desserts':'Bubble Tea',
    'Bubble Tea, Taiwanese':'Bubble Tea',
    'Bubble Tea, Tea, Beverages':'Bubble Tea',
    'Bubble Tea, Tea, Taiwanese, Beverages':'Bubble Tea',
    'Burger, Bar Food':'Burger',
    'Burger, Fast Food':'Fast Food',
    'Cafe Food, Bar Food':'Cafe Food',
    'Cafe Food, Coffee and Tea':'Cafe Food',
    'Cafe Food, Coffee and Tea, Mediterranean':'Cafe Food',
    'Cafe Food, Coffee and Tea, Modern Australian':'Cafe Food',
    'Cafe Food, Coffee and Tea, Vietnamese':'Cafe Food',
    'Cafe Food, Desserts, Coffee and Tea':'Cafe Food',
    'Cafe Food, Korean':'Cafe Food',
    'Chinese, Asian':'Chinese',
    'Chinese, BBQ':'Chinese',
    'Chinese, Cantonese, Coffee and Tea':'Chinese',
    'Chinese, Coffee and Tea':'Chinese',
    'Chinese, Dumplings':'Chinese',
    'Chinese, Dumplings, Hot Pot':'Chinese',
    'Chinese, Fast Food':'Fast Food',
    'Chinese, Hot Pot, Asian, BBQ, Cabinet Food':'Chinese',
    'Chinese, Hot Pot, Sichuan':'Chinese',
    'Chinese, Malatang':'Chinese',
    'Chinese, Malaysian':'Malaysian',
    'Chinese, Seafood':'Chinese',
    'Chinese, Sichuan':'Chinese',
    'Chinese, Street Food':'Chinese',
    'Chinese, Taiwanese':'Chinese',
    'Coffee and Tea':'Cafe Food',
    'Coffee and Tea, American':'Cafe Food',
    'Coffee and Tea, Beverages':'Cafe Food',
    'Coffee and Tea, Bubble Tea, Poké':'Bubble Tea',
    'Coffee and Tea, Cafe Food':'Cafe Food',
    'Coffee and Tea, Cafe Food, Sandwich':'Cafe Food',
    'Coffee and Tea, Desserts':'Cafe Food',
    'Coffee and Tea, Desserts, Frozen Yogurt':'Cafe Food',
    'Coffee and Tea, Healthy Food, Cafe Food':'Cafe Food',
    'Coffee and Tea, Healthy Food, Salad, Sandwich':'Cafe Food',
    'Coffee and Tea, Healthy Food, Sandwich':'Cafe Food',
    'Coffee and Tea, Italian':'Cafe Food',
    'Coffee and Tea, Modern Australian':'Cafe Food',
    'Coffee and Tea, Pizza, Cafe Food':'Cafe Food',
    'Coffee and Tea, Sandwich':'Cafe Food',
    'Coffee and Tea, Sandwich, Cafe Food':'Cafe Food',
    'Coffee and Tea, Sandwich, Desserts':'Cafe Food',
    'Coffee and Tea, Turkish':'Cafe Food',
    'Continental, Australian':'Australian',
    'Continental, Coffee and Tea, Cafe Food':'Cafe Food',
    'Deli, Coffee and Tea':'Cafe Food',
    'Desserts, Bakery':'Desserts',
    'Desserts, Beverages, Vietnamese':'Desserts',
    'Desserts, Cafe Food':'Desserts',
    'Desserts, Coffee and Tea':'Desserts',
    'Desserts, Coffee and Tea, Cafe Food':'Desserts',
    'Desserts, French, Coffee and Tea':'Desserts',
    'Desserts, Ice Cream':'Desserts',
    'Desserts, Modern Australian':'Desserts',
    'Desserts, Taiwanese':'Desserts',
    'Dumplings':'Chinese',
    'Dumplings, Chinese':'Chinese',
    'European, Bar Food':'Bar Food',
    'European, Modern Australian':'European',
    'European, Swiss':'European',
    'Fast Food, Fried Chicken':'Fast Food',
    'Fast Food, German, Eastern European':'Fast Food',
    'Fast Food, Japanese, Sushi':'Fast Food',
    'Fast Food, Taiwanese, Fried Chicken':'Fast Food',
    'French, Desserts, Coffee and Tea':'French',
    'Fried Chicken, Korean, Asian':'Korean',
    'Frozen Yogurt, Desserts':'Desserts',
    'Healthy Food, Cafe Food, Coffee and Tea, Sandwich':'Cafe Food',
    'Healthy Food, Fast Food, Salad':'Healthy Food',
    'Healthy Food, Juices':'Drinks Only',
    'Healthy Food, Salad, Sandwich':'Healthy Food',
    'Healthy Food, Vegetarian, Salad':'Healthy Food',
    'Hot Pot, Chinese, Asian':'Chinese',
    'Hot Pot, Sichuan, Chinese':'Chinese',
    'Ice Cream, Desserts, Coffee and Tea':'Desserts',
    'Indian, Sri Lankan':'Indian',
    'Indian, Vegetarian, Vegan':'Indian',
    'Indonesian, Malaysian':'Indonesian',
    'Italian, Bar Food':'Bar Food',
    'Italian, Cafe Food': 'Cafe Food',
    'Italian, Coffee and Tea, Cafe Food': 'Cafe Food',
    'Italian, European':'Italian',
    'Italian, Mediterranean':'Italian',
    'Italian, Pizza':'Italian',
    'Japanese, Australian, Asian':'Japanese',
    'Japanese, Beverages, Sushi':'Japanese',
    'Japanese, Cafe Food, Coffee and Tea':'Cafe Food',
    'Japanese, Desserts, Tea':'Desserts',
    'Japanese, Italian':'Japanese',
    'Japanese, Ramen':'Japanese',
    'Japanese, Sushi':'Japanese',
    'Japanese, Teppanyaki':'Japanese',
    'Juices, Beverages':'Drinks Only',
    'Juices, Cafe Food':'Drinks Only',
    'Kebab, Coffee and Tea':'Fast Food',
    'Korean BBQ, Korean':'Korean',
    'Korean, Asian':'Korean',
    'Korean, Chinese':'Korean',
    'Korean, Fried Chicken':'Korean',
    'Korean, Healthy Food, Fast Food, Asian':'Fast Food',
    'Korean, Korean BBQ':'Korean',
    'Malaysian, Coffee and Tea':'Cafe Food',
    'Modern Australian':'Australian',
    'Modern Australian, Burger, Bar Food':'Burger',
    'Modern Australian, Coffee and Tea':'Cafe Food',
    'Modern Australian, Coffee and Tea, Cafe Food': 'Cafe Food',
    'Modern Australian, Fusion, Asian':'Australian',
    'Modern Australian, Grill':'Australian',
    'Modern European':'European',
    'Modern European, Modern Australian':'European',
    'Patisserie':'Bakery',
    'Pho, Vietnamese, Asian':'Vietnamese',
    'Pizza, Italian':'Italian',
    'Pub Food, Modern Australian':'Pub Food',
    'Sandwich':'Cafe Food',
    'Sandwich, Cafe Food, Coffee and Tea':'Cafe Food',
    'Sandwich, Coffee and Tea':'Cafe Food',
    'Sandwich, Coffee and Tea, Cafe Food':'Cafe Food',
    'Sandwich, Fast Food':'Cafe Food',
    'Sandwich, Healthy Food, Coffee and Tea':'Cafe Food',
    'Sandwich, Vietnamese':'Vietnamese',
    'Seafood, Grill':'Seafood',
    'Seafood, Japanese':'Seafood',
    'Spanish, Bar Food, Cafe Food':'Cafe Food',
    'Spanish, Tapas':'European',
    'Sushi':'Japanese',
    'Sushi, Japanese, Korean':'Japanese',
    'Tea':'Drinks Only',
    'Tea, Bubble Tea, Beverages':'Bubble Tea',
    'Thai, Asian':'Thai',
    'Thai, Asian Fusion':'Thai',
    'Thai, BBQ, Hot Pot':'Thai',
    'Vegetarian, Coffee and Tea, Cafe Food':'Cafe Food',
    'Vietnamese, Asian, Fusion':'Vietnamese',
    'Vietnamese, Healthy Food':'Vietnamese',
    'Vietnamese, Pho':'Vietnamese',
    'Vietnamese, Pho, Asian':'Vietnamese',
    'Vietnamese, Sandwich':'Vietnamese'
    
})
list(z_clean_2.Type.sort_values().unique())

['American',
 'Asian',
 'Australian',
 'BBQ',
 'Bakery',
 'Bar Food',
 'Bubble Tea',
 'Burger',
 'Cafe Food',
 'Chinese',
 'Contemporary',
 'Desserts',
 'Drinks Only',
 'European',
 'Fast Food',
 'Fish and Chips',
 'French',
 'Hawaiian, Sushi',
 'Healthy Food',
 'Indian',
 'Indonesian',
 'Italian',
 'Japanese',
 'Korean',
 'Latin American, Peruvian',
 'Malatang',
 'Malaysian',
 'Meat Pie',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Nepalese',
 'Portuguese',
 'Pub Food',
 'Seafood',
 'Sri Lankan',
 'Thai',
 'Vietnamese']

In [15]:
# Save Zomato DataFrame (Unconsolidated Types) to CSV
z_clean_csv = z_clean.to_csv("Zomato_Restaurants_All.csv")
# Save Zomato DataFrame (Consolidated Types)
z_clean_types_csv = z_clean_2.to_csv("Zomato_Restaurants_All_Consolidated-Types.csv")

In [ ]:
# get coordinates of all relevant pedestrian traffic scanners

# add new column and bin data based on proximity to a sensor


In [ ]:
# Map locations of each restaurant onto Google Maps

# Compare with foot traffic

    # Make heatmap of foot traffic to see traffic volume

    # Assign ranking to foot traffic volume and add it as a numerical value to the Zomato DF as a new column
    
    # do a groupby